In [1]:
import pandas as pd

def convert_xlsx_to_csv(xlsx_file, csv_file):
    """
    从xlsx文件中读取前13列，保存为csv文件，所有数据存储为字符串格式。

    :param xlsx_file: 输入的xlsx文件路径
    :param csv_file: 输出的csv文件路径
    """
    # 读取Excel文件
    df = pd.read_excel(xlsx_file, dtype=str)  # 使用dtype=str将所有数据加载为字符串格式

    # 提取前13列
    df_selected = df.iloc[:, :13]

    # 将数据保存为CSV文件
    df_selected.to_csv(csv_file, index=False, quoting=1)  # quoting=1表示CSV中的内容用引号包裹

# 使用示例
xlsx_file = rf'C:\Users\dongz\Desktop\adni_dataset\label.xlsx'  # 输入Excel文件路径
csv_file = rf'C:\Users\dongz\Desktop\adni_dataset\ADNI.csv'  # 输出的CSV文件名

convert_xlsx_to_csv(xlsx_file, csv_file)


In [2]:
import os
import re

def rename_files_in_directory(directory, num_chars_to_remove):
    """
    重命名指定目录下的所有.nii文件，去掉文件名中开头的前num_chars_to_remove个字符，保留剩下的部分。
    
    :param directory: 要重命名文件的目录路径
    :param num_chars_to_remove: 要去掉的前几个字符的数量
    """
    # 获取指定目录下的所有文件
    files = os.listdir(directory)
    
    for file in files:
        # 检查文件是否是.nii文件
        if file.endswith('.nii'):
            # 去掉文件名开头的前num_chars_to_remove个字符
            new_name = file[num_chars_to_remove:]
            
            # 获取文件的完整路径
            old_path = os.path.join(directory, file)
            new_path = os.path.join(directory, new_name)
            
            # 重命名文件
            os.rename(old_path, new_path)
            print(f'Renamed: {file} -> {new_name}')
    
def main():
    """
    主函数，接收用户输入的文件根目录和去掉的前几个字符数
    """
    # 用户输入文件根目录和要去掉的前几个字符数
    directory_path = rf'C:\Users\dongz\Desktop\adni_dataset\MRI'
    num_chars_to_remove = 2
    
    # 调用重命名函数
    rename_files_in_directory(directory_path, num_chars_to_remove)
    
if __name__ == '__main__':
    main()


Renamed: 002_S_2010.nii -> 2_S_2010.nii
Renamed: 002_S_2043.nii -> 2_S_2043.nii
Renamed: 002_S_2073.nii -> 2_S_2073.nii
Renamed: 002_S_4171.nii -> 2_S_4171.nii
Renamed: 002_S_4213.nii -> 2_S_4213.nii
Renamed: 002_S_4219.nii -> 2_S_4219.nii
Renamed: 002_S_4225.nii -> 2_S_4225.nii
Renamed: 002_S_4237.nii -> 2_S_4237.nii
Renamed: 002_S_4251.nii -> 2_S_4251.nii
Renamed: 002_S_4262.nii -> 2_S_4262.nii
Renamed: 002_S_4270.nii -> 2_S_4270.nii
Renamed: 002_S_4447.nii -> 2_S_4447.nii
Renamed: 002_S_4473.nii -> 2_S_4473.nii
Renamed: 002_S_4521.nii -> 2_S_4521.nii
Renamed: 002_S_4654.nii -> 2_S_4654.nii
Renamed: 002_S_4746.nii -> 2_S_4746.nii
Renamed: 002_S_4799.nii -> 2_S_4799.nii
Renamed: 002_S_5018.nii -> 2_S_5018.nii
Renamed: 002_S_5178.nii -> 2_S_5178.nii
Renamed: 002_S_5230.nii -> 2_S_5230.nii
Renamed: 002_S_5256.nii -> 2_S_5256.nii
Renamed: 002_S_6652.nii -> 2_S_6652.nii
Renamed: 002_S_6695.nii -> 2_S_6695.nii
Renamed: 003_S_1057.nii -> 3_S_1057.nii
Renamed: 003_S_1059.nii -> 3_S_1059.nii


In [ ]:
'''生成表格数据'''


In [4]:
'''RID对应到Subject ID'''
import os
import re
import pandas as pd

# 配置路径
nii_dir = r'C:\Users\dongz\Desktop\adni_dataset\MRI'
csv_path = r'C:\Users\dongz\Desktop\adni_dataset\ADNI.csv'  # 请确认CSV文件路径

# 读取CSV文件
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    print(f"错误：CSV文件 {csv_path} 未找到")
    exit()

# 遍历NIFTI文件
for filename in os.listdir(nii_dir):
    if filename.endswith('.nii'):
        # 使用正则表达式提取数字部分
        match = re.search(r'_(\d+)\.nii$', filename)
        if not match:
            print(f"跳过文件 {filename}：格式不符合要求")
            continue
            
        # 提取并转换RID
        rid_str = match.group(1)
        try:
            rid = int(rid_str)  # 自动去除前导零
        except ValueError:
            print(f"文件 {filename} 的数字部分无效: {rid_str}")
            continue

        # 在CSV中查找匹配的Subject ID
        matched = df[df['RID'] == rid]
        if matched.empty:
            print(f"未找到RID为 {rid} 的记录，文件 {filename} 未处理")
            continue
            
        subject_id = matched.iloc[0]['Subject ID']
        new_filename = f"{subject_id}.nii"
        
        # 处理文件重命名
        src = os.path.join(nii_dir, filename)
        dst = os.path.join(nii_dir, new_filename)
        
        if os.path.exists(dst):
            print(f"文件 {new_filename} 已存在，跳过重命名")
            continue
            
        try:
            os.rename(src, dst)
            print(f"成功重命名: {filename} -> {new_filename}")
        except Exception as e:
            print(f"重命名失败: {str(e)}")

成功重命名: 0_S_0047.nii -> 100_S_0047.nii
成功重命名: 0_S_0067.nii -> 010_S_0067.nii
成功重命名: 0_S_0097.nii -> 020_S_0097.nii
成功重命名: 0_S_0161.nii -> 010_S_0161.nii
成功重命名: 0_S_0213.nii -> 020_S_0213.nii
成功重命名: 0_S_0232.nii -> 130_S_0232.nii
成功重命名: 0_S_0285.nii -> 130_S_0285.nii
成功重命名: 0_S_0289.nii -> 130_S_0289.nii
成功重命名: 0_S_0743.nii -> 100_S_0743.nii
成功重命名: 0_S_0747.nii -> 100_S_0747.nii
成功重命名: 0_S_0783.nii -> 130_S_0783.nii
成功重命名: 0_S_0786.nii -> 010_S_0786.nii
成功重命名: 0_S_0883.nii -> 020_S_0883.nii
成功重命名: 0_S_0892.nii -> 100_S_0892.nii
成功重命名: 0_S_0904.nii -> 010_S_0904.nii
成功重命名: 0_S_0930.nii -> 100_S_0930.nii
成功重命名: 0_S_0995.nii -> 100_S_0995.nii
成功重命名: 0_S_1062.nii -> 100_S_1062.nii
成功重命名: 0_S_1154.nii -> 100_S_1154.nii
成功重命名: 0_S_1290.nii -> 130_S_1290.nii
成功重命名: 0_S_2351.nii -> 100_S_2351.nii
成功重命名: 0_S_2373.nii -> 130_S_2373.nii
成功重命名: 0_S_2403.nii -> 130_S_2403.nii
成功重命名: 0_S_4345.nii -> 010_S_4345.nii
成功重命名: 0_S_4352.nii -> 130_S_4352.nii
成功重命名: 0_S_4405.nii -> 130_S_4405.nii
成功重命名: 0_S_4

In [5]:
'''重采样'''
import os
import SimpleITK as sitk

def resample_image(input_path, output_dir, target_size=(128, 128, 128), target_spacing=(2, 2, 2)):
    """
    重采样医学影像到指定尺寸和体素大小
    :param input_path: 输入影像路径（支持.nii/.nii.gz）
    :param output_dir: 输出目录
    :param target_size: 目标尺寸 (depth, height, width)
    :param target_spacing: 目标体素大小 (z, y, x) 单位：毫米
    """
    # 读取影像
    image = sitk.ReadImage(input_path)
    
    # 计算重采样参数
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(target_size)
    resampler.SetOutputSpacing(target_spacing)
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkLinear)  # 使用线性插值
    
    # 执行重采样
    resampled_image = resampler.Execute(image)
    
    # 保存结果
    filename = os.path.basename(input_path)
    output_path = os.path.join(output_dir, filename)
    sitk.WriteImage(resampled_image, output_path)
    print(f"Saved resampled image to: {output_path}")

# 使用示例
if __name__ == "__main__":
    # 输入路径和输出路径（替换为实际路径）
    mri_dir = rf"C:\Users\dongz\Desktop\adni_dataset\MRI"
    pet_dir = rf"C:\Users\dongz\Desktop\adni_dataset\PET"
    output_mri_dir = rf"C:\Users\dongz\Desktop\adni_dataset\MRI"  # MRI输出目录
    output_pet_dir = rf"C:\Users\dongz\Desktop\adni_dataset\PET"  # PET输出目录
    
    # 创建输出目录
    os.makedirs(output_mri_dir, exist_ok=True)
    os.makedirs(output_pet_dir, exist_ok=True)
    
    # 处理MRI影像，保存到output_mri_dir
    for filename in os.listdir(mri_dir):
        if filename.endswith(('.nii', '.nii.gz')):
            input_path = os.path.join(mri_dir, filename)
            resample_image(input_path, output_mri_dir)  # 指定MRI输出目录
    
    # 处理PET影像，保存到output_pet_dir
    for filename in os.listdir(pet_dir):
        if filename.endswith(('.nii', '.nii.gz')):
            input_path = os.path.join(pet_dir, filename)
            resample_image(input_path, output_pet_dir)  # 指定PET输出目录

Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_2010.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_2043.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_2073.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4171.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4213.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4219.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4225.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4237.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4251.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4262.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4270.nii
Saved resampled image to: C:\Users\dongz\Desktop\adni_dataset\MRI\002_S_4447.nii
Saved resampled image to: C: